In [49]:
import pandas as pd
import numpy as np
import requests
import json
import time

api_key = 'RGAPI-120a0c4f-daab-4bca-a742-a600ace56ffa'

In [50]:
def get_match_infos_from_id(matchId, api_key):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/BR1_{matchId}'
    header = {'api_key':api_key}
    r = requests.get(url, params=header)
    return r.json()

def get_id_from_url(matchUrl):
    match_details = matchUrl.split('#match-details/')[1]
    return match_details.split('/')[1]

def get_match_infos_from_url(matchUrl, api_key):
    matchId = get_id_from_url(matchUrl)
    return get_match_infos_from_id(matchId, api_key)

def get_list_of_ids_from_ulrs(list_of_urls):
    list_of_ids = []
    for url in list_of_urls:
        list_of_ids.append(get_id_from_url(url))
    return np.unique(list_of_ids)

def get_multiple_infos_from_urls(list_of_urls, api_key):
    list_of_infos = []
    for index, url in enumerate(np.unique(list_of_urls)):
        list_of_infos.append(get_match_infos_from_url(url, api_key))
        if index % 19 == 0:
            time.sleep(1)
    return list_of_infos

In [134]:

specific_matchId     = '2249128349'
specific_match_infos = get_match_infos_from_id(specific_matchId, api_key)

specific_macthUrl     = 'https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2249153501/200298930?tab=overview'
match_infos_Url      = get_match_infos_from_url(specific_macthUrl, api_key)

list_of_urls = [
    'https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2236441575/200298930?tab=overview',
    'https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2236441944/200298930?tab=overview',
    'https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2249153501/200298930?tab=overview',
    'https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2249153501/200298930?tab=overview',
    'https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2249128349/200298930?tab=overview',
    'https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2245262204/200298930?tab=overview'
    ]

list_of_infos = get_multiple_infos_from_urls(list_of_urls, api_key)

In [141]:
list_of_infos[1]

{'status': {'message': 'Data not found - match file not found',
  'status_code': 404}}

In [142]:
get_id_from_url('https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2236441575/200298930?tab=overview')

'2236441575'

In [144]:
get_match_infos_from_id('2236441575', api_key)

{'status': {'message': 'Data not found - match file not found',
  'status_code': 404}}

In [93]:
pd.DataFrame(data=list_of_infos[2]['info']['participants']).loc[:, ['puuid','championName','assists']]

,puuid,championName,assists
0,KHh6YCufvwNw9lvf4CUJgIh5geYYUmfGU-9x5HmajdVC-W...,Renekton,3
1,AlQSorxdyrS3tikba4tVuuJB39yexeGaCr7NJaxDnill3q...,Vi,3
2,Ui6aTQmpMKD053Zwb-RjJq4RaPKzyvUhBmFGyWu-mrJeNO...,Kassadin,3
3,w9yDDyIM39HxH-NTUp8rOSwdVaMd5YnGFDmoD4DPsnn_Qo...,Caitlyn,7
4,CwmjpQqJZ24HKLXVBWDoU0PVj8sOHdDeaNG_X8cPIypGjA...,Morgana,11
5,eA_5huR8D0_0CJmMUsUvFqCnoxZeJ3bQQfUZKTNHKNQekR...,Singed,15
6,bWt0RgcQs35IAt0LVTr9CEGvdes_Fyb8ETdmMuGaDgaqVm...,Olaf,8
7,2XL5NJqu3uV3Tp8FBFe5t3pBB3d8bR0xcbJ_PoaW-CQ7id...,Lux,11
8,pbw4bTAZfT85-XjtJ4K4S7mlQuJjVY5nVFoKNSriB_xpAz...,Jinx,8
9,nk4q-g5viAzmKyktV8m5mtNU03ZxucJzSL1m1zIbkxHeX6...,Zilean,13


In [146]:
def info_to_players_dataframe(match_infos):
    players = pd.DataFrame(match_infos['info']['participants'])
    players['gameId']             = match_infos['info']['gameId']
    players['gameDuration']       = match_infos['info']['gameDuration']
    players['gameStartTimestamp'] = match_infos['info']['gameStartTimestamp']

    return players

In [147]:
df = info_to_players_dataframe(specific_match_infos)
df

In [102]:
for column in a.columns:
    print(column)

assists
baronKills
bountyLevel
champExperience
champLevel
championId
championName
championTransform
consumablesPurchased
damageDealtToBuildings
damageDealtToObjectives
damageDealtToTurrets
damageSelfMitigated
deaths
detectorWardsPlaced
doubleKills
dragonKills
firstBloodAssist
firstBloodKill
firstTowerAssist
firstTowerKill
gameEndedInEarlySurrender
gameEndedInSurrender
goldEarned
goldSpent
individualPosition
inhibitorKills
inhibitorsLost
item0
item1
item2
item3
item4
item5
item6
itemsPurchased
killingSprees
kills
lane
largestCriticalStrike
largestKillingSpree
largestMultiKill
longestTimeSpentLiving
magicDamageDealt
magicDamageDealtToChampions
magicDamageTaken
neutralMinionsKilled
nexusKills
nexusLost
objectivesStolen
objectivesStolenAssists
participantId
pentaKills
perks
physicalDamageDealt
physicalDamageDealtToChampions
physicalDamageTaken
profileIcon
puuid
quadraKills
riotIdName
riotIdTagline
role
sightWardsBoughtInGame
spell1Casts
spell2Casts
spell3Casts
spell4Casts
summoner1Casts
su